Импортируем бибилотеки

In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


Отделим колонки и преобразовать вероятности в бинарные величины, где 1 — токсичный комментарий, а 0 — нетоксичный комментарий.

In [30]:
np.random.seed(0)

data = pd.read_csv("data/data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

Выведем данные, чтобы удостовериться, что все сделано верно

In [31]:
display(comments)
display(target)

0                     haha you guys are a bunch of losers.
1        Yet call out all Muslims for the acts of a few...
2        This bitch is nuts. Who would read a book by a...
3                                         You're an idiot.
4        Who cares!? Stark trek and Star Wars fans are ...
                               ...                        
90897    Methinks Bishop Braxton doth protest too much ...
90898    Sounds pretty speculative to me.  But i'm a sp...
90899    Seriously!\nVery proud of our 'domestic progra...
90900    Hawaii food is mostly GMO loaded with chemical...
90901    Eugenean:  I read hundreds and thousands of ar...
Name: comment_text, Length: 90902, dtype: object

0        1
1        1
2        1
3        1
4        1
        ..
90897    0
90898    0
90899    0
90900    0
90901    0
Name: target, Length: 90902, dtype: int64

Теперь разделим наши данные на train и test. Пусть в тест у нас пойдет 30% данных. 

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    comments, target, test_size=0.3, random_state=42
)

Преобразуем текст, который вы поделили на train и test, в числовой формат с помощью функции CountVectorizer.

In [33]:
vectorizer = CountVectorizer()
X_train_norm = vectorizer.fit_transform(X_train)
X_test_norm = vectorizer.transform(X_test)
print("Словарь:", vectorizer.get_feature_names_out())
print("Форма X_train_norm:", X_train_norm.shape)
print("Форма X_test_norm :", X_test_norm.shape)

Словарь: ['00' '000' '0000000000000000000' ... '𝒕𝒉𝒆' '𝒕𝒐' '𝓒𝓲𝓿𝓲𝓵']
Форма X_train_norm: (63631, 57878)
Форма X_test_norm : (27271, 57878)


в качестве модели, которая будет классифицировать нам комментарии на токсичные и нетоксичные, возьмем логистическую регрессию с параметром max_iter=2000. Для оценки модели возьмем метрику accuracy.

In [34]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train_norm, y_train)

# делаем предсказания на тесте
y_pred = model.predict(X_test_norm)

# считаем accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

Accuracy: 0.9279454365443145


Пропишем ниже функцию, для которой на вход мы бы подавали наш комментарий, а на выход получали предсказание, насколько от 0 до 1 комментарий является токсичным.

In [35]:
def predict_toxicity(comment: str) -> float:
    X_new = vectorizer.transform([comment])
    proba = model.predict_proba(X_new)[0][1]
    return proba

example = "Test comment"
score = predict_toxicity(example)
print(f"Вероятность токсичности: {score:.3f}")

Вероятность токсичности: 0.120


Попробуем предсказать, токсичен ли комментарий «Apples are stupid». Потом предскажем, токсичен ли комментарий «I love apples».

In [36]:
toxic_comment = 'Apples are stupid'
non_toxic_comment = 'I love apples'

score_toxic = predict_toxicity(toxic_comment)
score_non_toxic = predict_toxicity(non_toxic_comment)

print(f"Вероятность токсичности токсичного комментрия: {score_toxic:.3f}")
print(f"Вероятность токсичности нетоксичного комментрия: {score_non_toxic:.3f}")


Вероятность токсичности токсичного комментрия: 0.999
Вероятность токсичности нетоксичного комментрия: 0.058


Выведем десять слов, которые считаются наиболее токсичными, а также их коэффициенты.


In [37]:
vocab = vectorizer.vocabulary_
coefs = model.coef_[0]

word_weights = [
    (word, coefs[vocab[word]])
    for word in vocab.keys()
]

top10 = sorted(word_weights, key=lambda x: x[1], reverse=True)[:10]
for word, weight in top10:
    print(f"{word}: {weight:.4f}")

stupid: 9.1988
idiot: 8.7349
idiots: 8.4570
stupidity: 7.5453
idiotic: 6.8279
crap: 6.5746
dumb: 6.4499
pathetic: 6.4234
hypocrite: 6.3885
moron: 6.3644


Взгляните на самые токсичные слова из задания 6. Вызывают ли у вас удивление какие-нибудь из них? Есть ли слова, которых, кажется, не должно быть в списке?

Ответ:

На первый взгляд, лишних слов нет. Возможно стоит обратить внимание на слово "stupidity", иак как слово "глупость" само по себе не является токсичным и может быть использовано в не токсичном контекстею

Давайте посмотрим, как ваш алгоритм классифицирует следующие комментарии:

"I have a christian friend"
"I have a muslim friend"
"I have a white friend"
"I have a black friend"

In [38]:
test_phrases = ["I have a christian friend",
"I have a muslim friend",
"I have a white friend",
"I have a black friend"]

for phrase in test_phrases:
    score = predict_toxicity(phrase)
    print(f'Фраза "{phrase}" является токсичной с вероятностью {score}')



Фраза "I have a christian friend" является токсичной с вероятностью 0.18648912185234992
Фраза "I have a muslim friend" является токсичной с вероятностью 0.512137300359607
Фраза "I have a white friend" является токсичной с вероятностью 0.4045665156940112
Фраза "I have a black friend" является токсичной с вероятностью 0.5882626923835796


Что думаете о получившихся результатах? Есть ли у модели bias? Этичен ли он?


Ответ:
У модели действительно есть предвзятость по рассовому и религиозному признаку.

Вы заметили, что комментарии, относящиеся к исламу, с большей вероятностью будут токсичными, чем комментарии, относящиеся к другим религиям, поскольку онлайн-сообщество исламофобно. Какой тип предвзятости это может внести в вашу модель?

Ответ:
 
Обычно такой тип предвзятости называют предвзяточтью выборки, так как в выборке было больше токсичных комментариев с использованием конкретных расс и религий.

Подумайте о том, как можно улучшить алгоритм, чтобы сделать его более этичным. Напишите 1–2 идеи.

Ответ:

Во-первых, можно было бы уровновесить упоминание различных расс и религий как в комментариях, которые считаются токсичными, так и в комментриях, которые являются нетоксичными. 

Во-вторых, можно было бы в целом убрать примеры, которые затрагивают какие-то аспекты чувствительных тем, но тогда модель могла бы не учитывать определенный контекст, так как в выборке бы отсутствовали примеры с таким содержанием 